<a href="https://colab.research.google.com/github/cemvardar/pragmatic_google_colab/blob/main/dslab_colab_utility_shopify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
import requests
import json

In [2]:
class Product:
    def __init__(self, shopify_data):
        self.shopify_data = shopify_data
        self.title = self.shopify_data['title']
        self.product_type = self.shopify_data['product_type']
        self.body_html = self.shopify_data['body_html']
        self.tags = self.shopify_data['tags']
        self.id = self.shopify_data['id']
        self.handle = f"https://www.liplips.co.uk/products/{self.shopify_data['handle']}"
        self.admin_handle = f"https://admin.shopify.com/store/liplipsstore/products/{self.id}"
        self.status = self.shopify_data['status']
        self.is_active = self.status=='active'


class Collection():
    def __init__(self, shopify_data):
        self.shopify_data = shopify_data
        self.title = self.shopify_data['title']
        self.body_html = self.shopify_data['body_html']
        self.id = self.shopify_data['id']
        # self.collects = get_shopify_data('collects', ('collection_id', self.id))['collects']
        self.collects = None
        self.handle = f"https://www.liplips.co.uk/collections/{self.shopify_data['handle']}"
        self.admin_handle = f"https://admin.shopify.com/store/liplipsstore/collections/{self.id}"

    def products_list(self):
        products_list = []
        if self.collects == None:
            self.collects = get_shopify_data('collects', ('collection_id', self.id))['collects']
        for collect in self.collects:
            products_list.append(products_dict[collect['product_id']])
        return products_list

    def get_product_image_url(self, product_index):
        products = self.products_list()
        return products[product_index].shopify_data['images'][0]['src']

    def get_video_file_name(self):
        return f"{self.title.replace(' ', '_')}.mp4"


def get_api_url(element, query_tuple=None):
    url_template = f'https://liplipsstore.myshopify.com/admin/api/2024-04/{element}.json'
    if query_tuple is not None:
        query_suffix = f'?{query_tuple[0]}={query_tuple[1]}'
        return url_template+query_suffix
    return url_template


def get_shopify_data(element, query_tuple=None, fields=None):
    access_token = userdata.get('liplips_shopify_token')

    # Set up the headers with the necessary authentication
    headers = {
        "Content-Type": "application/json",
        "X-Shopify-Access-Token": access_token
    }
    params = {'limit': 250}  # Set initial limit to 250
    if fields is not None:
        params['fields'] = fields
    # Define the endpoint for fetching products
    url = get_api_url(element, query_tuple)

    all_items = []  # Initialize an empty list to store all items
    while url:  # Loop until there are no more pages
        response = requests.get(url, headers=headers, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            all_items.extend(data[element])  # Add items from current page

            # Get the link for the next page, if available
            link_header = response.headers.get('Link')
            if link_header:
                links = link_header.split(',')
                for link in links:
                    if 'rel="next"' in link:
                        url = link.split(';')[0].strip('<>')
                        break  # Found the next page link
                else:
                    url = None  # No next page link found
            else:
                url = None  # No Link header, assume no more pages

        else:
            print(f"Failed to retrieve {element}: {response.status_code}")
            break  # Exit the loop on error

    return {element: all_items}  # Return all fetched items

def get_collection_dict():
    element = 'custom_collections'
    collections = get_shopify_data(element)[element]
    print(f'{len(collections)} collections')
    collection_dict = {}
    for collection_data in collections:
        collection = Collection(collection_data)
        collection_dict[collection.id] = collection

    return collection_dict


def get_product_dict():
    element = 'products'
    products = get_shopify_data(element)[element]

    print(f'{len(products)} products')

    products_dict = {}
    for product_data in products:
        product = Product(product_data)
        products_dict[product.id] = product

    return products_dict

products_dict = get_product_dict()
collection_dict = get_collection_dict()
products = list(products_dict.values())
collections = list(collection_dict.values())

328 products
111 collections


In [5]:
def update_product_title_and_description(product_id: int, title: str = None, body_html: str = None) -> dict:
    """
    Update a product's title and/or body HTML in Shopify.

    Args:
        product_id (int): The ID of the product to update
        title (str, optional): New title for the product
        body_html (str, optional): New body HTML for the product

    Returns:
        dict: The response from Shopify API containing the updated product data
    """
    access_token = userdata.get('liplips_shopify_token')

    headers = {
        "Content-Type": "application/json",
        "X-Shopify-Access-Token": access_token
    }

    # Prepare update data
    update_data = {"product": {"id": product_id}}
    if title is not None:
        update_data["product"]["title"] = title
    if body_html is not None:
        update_data["product"]["body_html"] = body_html

    # Make the API call
    url = f'https://liplipsstore.myshopify.com/admin/api/2024-04/products/{product_id}.json'
    response = requests.put(url, headers=headers, json=update_data)

    if response.status_code == 200:
        print(f"Product {product_id} updated successfully")
        return response.json()
    else:
        error_msg = f"Failed to update product {product_id}. Status code: {response.status_code}, Response: {response.text}"
        print(error_msg)
        raise Exception(error_msg)

# test_product_id = 14935728259455
# response = update_product_title_and_description(product_id,  body_html= 'cem')

Product 14935728259455 updated successfully


In [ ]:
def add_product_to_shopify(product_data):
    # Headers for authentication and content type
    headers = {
        "Content-Type": "application/json",
        "X-Shopify-Access-Token": userdata.get('liplips_shopify_token')
    }

    # Data to be updated
    data = {
        "product": product_data
    }

    # API endpoint to update the product
    url = f'https://liplipsstore.myshopify.com/admin/api/2023-01/products.json'

    # Make the PUT request to update the product description
    response = requests.post(url, headers=headers, json=data)

    # Check the response
    if response.status_code == 201:
        print("Product description updated successfully.")
        updated_product = response.json()
        print(f'''https://admin.shopify.com/store/liplipsstore/products/{updated_product['product']['id']}''')
    else:
        print(f"Failed to update product: {response.status_code} - {response.text}")
    return updated_product

# product_data = {
#             "title": "aaaa test",
#             "vendor": "Liplips",
#             "body_html": 'testing',
#             "product_type": "Necklaces",
#             'status': 'draft',
#             "variants": [
#                 {
#                     "option1": "Default Title",
#                     "price": 10,
#                     "sku": 'rs_test',
#                     "inventory_management": "shopify",
#                     "inventory_quantity": 5
#                 }
#                 ],
#             'images': []
#             }

# updated_product = add_product_to_shopify(product_data)


In [ ]:
def get_blogs_and_posts():
    # Get blogs
    blogs = get_shopify_data('blogs')['blogs']
    all_blog_posts = []
    # Loop over blogs
    for blog in blogs:
        # Get blog posts for each blog
        blog_posts = get_shopify_data('articles', ('blog_id', blog['id']))['articles']
        all_blog_posts.extend(blog_posts)
    return blogs, all_blog_posts

blogs, blog_posts = get_blogs_and_posts()
# print(f"Number of blogs: {len(blogs)}")
# print(f"Number of blog posts: {len(blog_posts)}")

In [ ]:
def get_metafields_for_resource(resource_type, resource_id):
    access_token = userdata.get('liplips_shopify_token')

    # Set up the headers with the necessary authentication
    headers = {
        "Content-Type": "application/json",
        "X-Shopify-Access-Token": access_token
    }
    # Define the endpoint for fetching metafields
    url = f'https://liplipsstore.myshopify.com/admin/api/2023-01/{resource_type}/{resource_id}/metafields.json'

    # Make the HTTP GET request to the Shopify API
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Load the JSON data from the response
        metafields = response.json()['metafields']
        return metafields

    else:
        print(f"Failed to retrieve metafields: {response.status_code}")
        return None

# Example usage:
# blog_post_id = 558719139882  # Replace with the actual blog post ID
# metafields = get_metafields_for_resource('articles', blog_post_id)
# metafields
metafields = get_metafields_for_resource('products', 6992724754474)
metafields

[{'id': 25482829561898,
  'namespace': 'mm-google-shopping',
  'key': 'age_group',
  'value': 'adult',
  'description': None,
  'owner_id': 6992724754474,
  'created_at': '2023-11-01T01:08:04+00:00',
  'updated_at': '2023-11-01T01:08:04+00:00',
  'owner_resource': 'product',
  'type': 'string',
  'admin_graphql_api_id': 'gid://shopify/Metafield/25482829561898'},
 {'id': 25482829594666,
  'namespace': 'mm-google-shopping',
  'key': 'color',
  'value': 'Multi',
  'description': None,
  'owner_id': 6992724754474,
  'created_at': '2023-11-01T01:08:04+00:00',
  'updated_at': '2023-11-01T01:08:04+00:00',
  'owner_resource': 'product',
  'type': 'string',
  'admin_graphql_api_id': 'gid://shopify/Metafield/25482829594666'},
 {'id': 25482829627434,
  'namespace': 'mm-google-shopping',
  'key': 'condition',
  'value': 'new',
  'description': None,
  'owner_id': 6992724754474,
  'created_at': '2023-11-01T01:08:04+00:00',
  'updated_at': '2023-11-01T01:08:04+00:00',
  'owner_resource': 'product',


In [ ]:
def update_metafield(resource_type, resource_id, data):
    access_token = userdata.get('liplips_shopify_token')

    # Set up the headers with the necessary authentication
    headers = {
        "Content-Type": "application/json",
        "X-Shopify-Access-Token": access_token
    }

    url = f'https://liplipsstore.myshopify.com/admin/api/2023-01/{resource_type}/{resource_id}/metafields.json'
    response = requests.post(url, headers=headers, json=data)

    if (response.status_code == 200) or (response.status_code == 201):
        print(f"Metafield updated successfully. HTML code {response.status_code}")
    else:
        print(f"Failed to update metafield: {response.status_code}, {response.text}")

# new_value = "gid://shopify/Collection/280784896042"
# data = {
#     "metafield": {'namespace': 'custom',
#                   'key': 'blog_collection2',
#                   'value': new_value
#     }
# }
# update_metafield('articles' , 558719139882, data)


# data = {
#     "metafield": {
#                 'namespace': 'global',
#                 'key': 'title_tag',
#                 'value': 'Flower-inspired Jewellery | Liplips Studio'
#                 }
# }
# update_metafield('custom_collections', 280707760170, data)
data = {
    "metafield": {
                'namespace': 'mm-google-shopping',
                'key': 'google_product_category',
                'value': '191'
                }
}
update_metafield('products', 6992724754474, data)

Metafield updated successfully. HTML code 201


In [ ]:
def add_to_shopify(element_type, element_data):
    # Headers for authentication and content type
    headers = {
        "Content-Type": "application/json",
        "X-Shopify-Access-Token": userdata.get('liplips_shopify_token')
    }

    # API endpoint to add the element
    url = f'https://liplipsstore.myshopify.com/admin/api/2023-01/{element_type}.json'

    # Data to be added, wrapped in the appropriate key
    data = {
        element_type[:-1]: element_data  # Remove the trailing 's' from element_type
    }

    # Make the POST request
    response = requests.post(url, headers=headers, json=data)

    # Check the response
    if response.status_code == 201:
        print(f"{element_type[:-1]} added successfully.")
        added_element = response.json()
        print(f'''https://admin.shopify.com/store/liplipsstore/{element_type}/{added_element[element_type[:-1]]['id']}''')
    else:
        print(f"Failed to add {element_type[:-1]}: {response.status_code} - {response.text}")
    return added_element

In [ ]:
# article_data = {
#     "title": "New Summer Collection Launch",
#     "author": "Liplips Team",
#     "blog_id": 84788740138,  # Replace with actual blog ID
#     "body_html": "<h2>Get ready for summer with our latest collection!</h2><p>Check out our new vibrant colors and styles...</p>",
#     "tags": "summer, new collection, fashion",
#     'published_at': None,
#     'summary_html': "excerpt sample",
#     "metafields": [
#         {
#             "namespace": "custom",
#             "key": "blog_collection2",
#             "value": "gid://shopify/Collection/280784896042",
#             "value_type": "string"
#         },
#         {
#             "namespace": "custom",
#             "key": "blog_collection_1",
#             "value": "gid://shopify/Collection/280784896042",
#             "value_type": "string"
#         }
#     ]
# }
# added_article = add_to_shopify("articles", article_data)

article added successfully.
https://admin.shopify.com/store/liplipsstore/articles/558721007658
